In [1]:
import pymongo
import time
import threading
import random
from pymongo.collection import Collection

In [2]:
cl = pymongo.MongoClient("mongodb://localhost:27017")
db = cl["deep"]
arr1 = db["arr1"]
arr2 = db["arr2"]
arr4 = db["arr4"]
arr8 = db["arr8"]
coll_list = [arr1, arr2, arr4, arr8]

In [3]:
user_list = [1, 2, 4]
depth_list = [1, 2, 4, 8]

In [104]:
new_docs_size = 500

In [4]:
def create_batch(size: int, coll: Collection):
    new_objs = list(coll.find({}, {"_id": 0}).limit(size))
    for el in new_objs:
        el["inserted"] = True
    return new_objs

In [85]:
def measure_insert(user_id, duration_list, coll):
    duration = 0
    tries = 5
    for t in range(tries):
        batch = create_batch(new_docs_size, coll)
        start = time.perf_counter()
        coll.insert_many(batch)
        end = time.perf_counter()
        duration += (end-start)
    
    duration = (duration / tries) * 1000
    duration_list[user_id] = duration        

In [29]:
def test_insert(num_of_users, depth):
    threads_list = []
    duration_list = [None] * num_of_users
    coll = coll_list[depth]

    for x in range(num_of_users):
        t = threading.Thread(target=measure_insert, args=(x, duration_list, coll))
        t.start()
        threads_list.append(t)

    for t in threads_list:
        t.join()

    result = sum(duration_list)/len(duration_list)
    return result

In [80]:
def execute_insert():
    u_len = len(user_list)
    d_len = len(depth_list)
    result = [[0 for _ in range(d_len)] for _ in range(u_len)]

    for u in range(u_len):
        print(f"user {user_list[u]}")
        print("depth: ", end="")
        for d in range(d_len):
            result[u][d] = test_insert(user_list[u], d)
            print(f"{depth_list[d]}", end=" ")
        print("---")            
    return result

In [36]:
def calculate_diffs(old, new):
    for u in range(len(user_list)):
        print(f"user {user_list[u]}")
        for d in range(len(depth_list)):
            o_val = old[u][d]
            n_val = new[u][d]
            diff = ((n_val - o_val) / o_val) * 100
            print(f"| {depth_list[d]} |{o_val:10.0f}| {n_val:10.0f}| {diff:10.1f} |")
        print("--------")

In [37]:
def calc_diff_precise(old, new):
    for u in range(len(user_list)):
        print(f"user {user_list[u]}")
        for d in range(len(depth_list)):
            o_val = old[u][d]
            n_val = new[u][d]
            diff = ((n_val - o_val) / o_val) * 100
            print(f"depth: {depth_list[d]}, {o_val:10.2f} {n_val:10.2f} {diff:10.2f}")
        print("--------")

In [21]:
def createIndexes():
    coll_list[0].create_index([("a1.device", 1)])
    coll_list[1].create_index([("a1.a2.device", 1)])
    coll_list[2].create_index([("a1.a2.a3.a4.device", 1)])
    coll_list[3].create_index([("a1.a2.a3.a4.a5.a6.a7.a8.device", 1)])

In [22]:
def clearIndexes():
    for i in coll_list:
        i.drop_indexes()

In [23]:
def showIndexes():
    for i in arr1.list_indexes():
        print(i)
    for a in arr2.list_indexes():
        print(a)
    for b in arr4.list_indexes():
        print(b)
    for c in arr8.list_indexes():
        print(c)

In [57]:
def clean_up():
    for c in coll_list:
        print(c.delete_many({"inserted": True}))

In [87]:
def count_inserted():
    for c in coll_list:
        print(c.count_documents({"inserted": True}))

In [91]:
showIndexes()

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])


In [74]:
clearIndexes()

In [92]:
without_index = execute_insert()

user 1
depth: 1 2 4 8 ---
user 2
depth: 1 2 4 8 ---
user 4
depth: 1 2 4 8 ---


In [96]:
clean_up()

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)
DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)


In [95]:
createIndexes()

In [97]:
with_index = execute_insert()

user 1
depth: 1 2 4 8 ---
user 2
depth: 1 2 4 8 ---
user 4
depth: 1 2 4 8 ---


In [98]:
calculate_diffs(without_index, with_index)

user 1
| 1 |         8|         13|       65.5 |
| 2 |         8|         19|      137.3 |
| 4 |        15|         45|      198.1 |
| 8 |       143|        662|      362.2 |
--------
user 2
| 1 |        10|         17|       70.9 |
| 2 |        12|         21|       75.4 |
| 4 |        36|         80|      125.3 |
| 8 |       576|       1213|      110.6 |
--------
user 4
| 1 |        12|         17|       42.1 |
| 2 |        16|         33|      106.7 |
| 4 |        71|        117|       63.9 |
| 8 |      1822|       2718|       49.2 |
--------


In [100]:
clearIndexes()